## Gameplan for this bs

1. Run Vina on all scPDB files and see output quality

In [1]:
import os
os.chdir('/home/longyuxi/Documents/algebraic-topology-biomolecules/vina-benchmark/scpdb-example/1a2b_1')
adfr_root = '/home/longyuxi/ADFRsuite-1.0/bin'

In [2]:
# Prepare the receptor
assert os.system(f'obabel protein.mol2 -O protein.pdb') == 0
assert os.system(f'{adfr_root}/prepare_receptor -r protein.pdb -o protein.pdbqt') == 0

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1a2b_GSP_1_protein)

1 molecule converted


Sorry, there are no Gasteiger parameters available for atom protein:A:THR16:OG1
Sorry, there are no Gasteiger parameters available for atom protein:A:THR34:OG1
Unable to assign MAP type to atom Mg
Sorry, there are no Gasteiger parameters available for atom protein:A:MG5179:MG


In [3]:
# Prepare the ligand
assert os.system('mk_prepare_ligand.py -i ligand.sdf -o ligand.pdbqt') == 0

In [4]:
# Calculate bounding box
from utils import calculate_bbox
import pybel

mols = list(pybel.readfile('mol2', 'protein.mol2'))
assert len(mols) == 1
mol = mols[0]
bbox_center, bbox_lengths = calculate_bbox(mol)

with open('vina_box.txt', 'w') as txt:
    txt.write(f'center_x = {bbox_center[0]}\ncenter_y = {bbox_center[1]}\ncenter_z = {bbox_center[2]}\nsize_x = 20.0\nsize_y = 20.0\nsize_z = 20.0')


*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1a2b_GSP_1_protein)



In [14]:
cmd = f'vina --receptor protein.pdbqt --ligand ligand.pdbqt \
       --config vina_box.txt \
       --exhaustiveness=32 --out ligand_vina_out.pdbqt'

# cmd = f'vina --receptor protein.pdbqt --ligand ligand.pdbqt \
#        --autobox \
#        --exhaustiveness=32 --out ligand_vina_out.pdbqt'

assert os.system(cmd) == 0

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [17]:
# Compare against grouth truth
from utils import calculate_rmsd

gt = list(pybel.readfile('pdbqt', 'ligand.pdbqt'))
assert len(gt) == 1
gt = gt[0]

# print(len(list(pybel.readfile('pdbqt', 'ligand_vina_out.pdbqt'))))

with open('vina_prediction_rmsds.txt', 'w') as f:
    for prediction in pybel.readfile('pdbqt', 'ligand_vina_out.pdbqt'):
        f.write(f'{calculate_rmsd(gt, prediction)}\n')